In [3]:
import pandas as pd
import os

In [5]:
# Get the current working directory
current_path = os.getcwd()
print("Current Path:", current_path)

# Set a new working directory
base_path = '/Users/yifanli/Github/fidelity-portfolio-tracker'
os.chdir(base_path)


Current Path: /Users/yifanli/Github/fidelity-portfolio-tracker/code


In [16]:
file_path = './data/Accounts_History_2022Q3.csv'
dat_ori = pd.read_csv(file_path)
dat = dat_ori[dat_ori['Account'].notna()]

In [17]:
dat

,Run Date,Account,Action,Symbol,Description,Type,Quantity,Price ($),Commission ($),Fees ($),Accrued Interest ($),Amount ($),Settlement Date
0,09/30/2022,Individual Z23390746,REINVESTMENT FIDELITY TREASURY MONEY MARKET F...,FZFXX,FIDELITY TREASURY MONEY MARKET FUND,Cash,34.02,1.00,NaN,NaN,NaN,-34.02,NaN
1,09/30/2022,Cash Management (Individual) Z06872898,INTEREST EARNED FDIC INSURED DEPOSIT AT US BA...,QUSBQ,FDIC INSURED DEPOSIT AT US BANK NOT COV,Cash,0.00,NaN,NaN,NaN,NaN,0.10,NaN
2,09/30/2022,Individual Z23390746,DIVIDEND RECEIVED FIDELITY TREASURY MONEY MAR...,FZFXX,FIDELITY TREASURY MONEY MARKET FUND,Cash,0.00,NaN,NaN,NaN,NaN,34.02,NaN
3,09/01/2022,Individual Z23390746,YOU BOUGHT AMERICAN EXPRESS CO COM USD0.20 (A...,AXP,AMERICAN EXPRESS CO COM USD0.20,Cash,10.00,150.00,NaN,NaN,NaN,-1500.00,09/06/2022
4,08/31/2022,Individual Z23390746,REINVESTMENT FIDELITY TREASURY MONEY MARKET F...,FZFXX,FIDELITY TREASURY MONEY MARKET FUND,Cash,32.21,1.00,NaN,NaN,NaN,-32.21,NaN
5,08/31/2022,Individual Z23390746,DIVIDEND RECEIVED FIDELITY TREASURY MONEY MAR...,FZFXX,FIDELITY TREASURY MONEY MARKET FUND,Cash,0.00,NaN,NaN,NaN,NaN,32.21,NaN
6,08/31/2022,Cash Management (Individual) Z06872898,INTEREST EARNED FDIC INSURED DEPOSIT AT US BA...,QUSBQ,FDIC INSURED DEPOSIT AT US BANK NOT COV,Cash,0.00,NaN,NaN,NaN,NaN,0.05,NaN
7,08/29/2022,Individual Z23390746,YOU BOUGHT APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,20.00,160.00,NaN,NaN,NaN,-3200.00,08/31/2022
8,08/26/2022,Individual Z23390746,DIVIDEND RECEIVED STARBUCKS CORP COM USD0.001...,SBUX,STARBUCKS CORP COM USD0.001,Cash,0.00,NaN,NaN,NaN,NaN,4.90,NaN
9,08/26/2022,Individual Z23390746,YOU BOUGHT APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,10.00,164.50,NaN,NaN,NaN,-1645.00,08/30/2022
